## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

pd.options.mode.chained_assignment = None  # default='warn'

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_2000Cases_New/'
# TestDir    = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_Test/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

NVarsRed           = 7

tStratch           = 1.
SOLVER             = 'BDF'

valid_perc          = 20.

FixedMinVal        = 1.e-14


NNew               = 1000

In [3]:
Vars = []
for iVars in range(1,NVarsRed+1):
    Vars.append('PC_'+str(iVars))

Vars0 = []
for iVars in range(1,NVarsRed+1):
    Vars0.append('PC0_'+str(iVars))
    
SVars = []
for iVars in range(1,NVarsRed+1):
    SVars.append('SPC_'+str(iVars))
    

FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
n_ics         = len(ICVecs)


P0Vec        = []
EqRatio0Vec  = []
T0Vec        = []
for iC in range(n_ics):
    Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
    P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
    EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
    T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]

In [4]:
# for iC in range(n_ics):
    
#     Nts                = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName           = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataRead           = pd.read_csv(FileName, header=0)

#     indxs              = np.random.randint(0, high=len(DataRead), size=(NNew,2), dtype=int)
#     indxs              = np.sort(indxs)
#     DataSampled        = DataRead[Vars].loc[indxs[:,1]].reset_index(drop=True)
#     DataSampled[Vars0] = DataRead[Vars].loc[indxs[:,0]].reset_index(drop=True)
#     DataSampled['t']   = DataRead['t'].loc[indxs[:,1]].to_numpy() - DataRead['t'].loc[indxs[:,0]].to_numpy()  
    
#     Data0              = DataRead.iloc[[0]][Vars]
#     Data0              = Data0.loc[Data0.index.repeat(Nts)]
#     Data0.reset_index(drop=True, inplace=True)
#     DataRead[Vars0]    = Data0
    
#     if (iC==0):
#         DataAll        = pd.concat([DataRead, DataSampled], axis=0)
#     else:
#         DataAll        = pd.concat([DataAll, DataRead, DataSampled], axis=0)
        
# DataAll = DataAll.reset_index(drop=True)

In [5]:
for iC in range(n_ics):
    
    Nts                = SimIdxs[iC,1]-SimIdxs[iC,0]
    FileName           = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
    DataRead           = pd.read_csv(FileName, header=0)

    indxs              = np.random.randint(0, high=len(DataRead), dtype=int)
    DataSampled        = DataRead[Vars].reset_index(drop=True)
    DataSampled[Vars0] = DataRead[Vars].loc[indxs].reset_index(drop=True)
    DataSampled['t']   = DataRead['t'].to_numpy() - DataRead['t'].loc[indxs]
    DataSampled        = DataSampled.drop(DataSampled[DataSampled['t'] < 1.e-10].index)
    
    Data0              = DataRead.iloc[[0]][Vars]
    Data0              = Data0.loc[Data0.index.repeat(Nts)]
    Data0.reset_index(drop=True, inplace=True)
    DataRead[Vars0]    = Data0
    
#     if (iC==0):
#         DataAll        = pd.concat([DataSampled], axis=0)
#     else:
#         DataAll        = pd.concat([DataAll, DataSampled], axis=0)
    if (iC==0):
        DataAll        = pd.concat([DataRead, DataSampled], axis=0)
    else:
        DataAll        = pd.concat([DataAll, DataRead, DataSampled], axis=0)
        
DataAll = DataAll.reset_index(drop=True)

In [6]:
#DataAll.iloc[DataAll[DataAll['t'] < 1.e-14].index]
#DataAll[DataAll['t'] < 1.e-14]
#DataAll[DataAll['t'] < 1.e-14].index
#DataAll.index

In [7]:
# plt.plot(DataAll['t'], DataAll['PC_1'], 'o')
# plt.xscale('log')

In [8]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/')
except:
    pass

In [9]:
data_id    = 'pts'

n_points             = len(DataAll)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput  = DataAll[['t'] + Vars0]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataAll[['t'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [10]:
#pd.plotting.scatter_matrix(DataAll[Vars0], alpha=0.2)